This script will export rasters which calculate the om/com from gabam and us comparted to mtbs and nbac

In [1]:
import os
import ee
import numpy as np
from geeml.extract import extractor
import pandas as pd
import geemap
import os
from google.cloud import storage
from google.cloud import client
# import geemap
# Authenticate GEE
# ee.Authenticate()
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/explore/nobackup/people/spotter5/cnn_mapping/gee-serdp-upload-7cd81da3dc69.json"

service_account = 'gee-serdp-upload@appspot.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, "/explore/nobackup/people/spotter5/cnn_mapping/gee-serdp-upload-7cd81da3dc69.json")
ee.Initialize(credentials)
# Initialize GEE with high-volume end-point
# ee.Initialize(opt_url='https=#earthengine-highvolume.googleapis.com')
ee.Initialize()


os.environ["GCLOUD_PROJECT"] = "gee-serdp-upload"
storage_client = storage.Client()
# bucket_name = 'smp-scratch/mtbs_1985'
bucket_name = 'smp-scratch'

bucket = storage_client.bucket(bucket_name)

In [2]:
#year of interest
year = 2015

# lfdb = ee.FeatureCollection("users/spotter/fire_cnn/raw/ak_ca_1985")
# # aoi2 = ee.FeatureCollection("users/spotter/fire_cnn/raw/ak_ca_west")
# # t3 = ee.ImageCollection("users/spotter/fire_cnn/mtbs_predictions/ak_compare_64_32")
# # table = ee.FeatureCollection("users/spotter/fire_cnn/raw/ak_cent_grid")
# viirs = ee.Image("users/spotter/fire_cnn/ak_ca_viirs_2015")
# # ak = ee.FeatureCollection("users/spotter/alaska")
# # sk = ee.FeatureCollection("users/spotter/SK_boundary")
# buffer = ee.FeatureCollection("users/spotter/fire_cnn/buffered_polys/2015")
# water = ee.ImageCollection("JRC/GSW1_4/YearlyHistory")
# # grd =  ee.FeatureCollection("users/spotter/fire_cnn/raw/ak_ca_grd_west_largest3")
territory22 = ee.FeatureCollection("users/spotter/fire_cnn/raw/ak_cent_grid")
# predictions = ee.ImageCollection('users/spotter/fire_cnn/mtbs_predictions/ak_compare_64_32').max()

# water = water.filterDate('2015-01-01', '2015-12-31').max().select('waterClass').clip(territory22.union())
# water = water.updateMask(water.eq(3))

# #modis land cover for now
# lc = ee.ImageCollection("MODIS/061/MCD12Q1")

# #read in gabam which is a 30m random forest model
# gabam = ee.ImageCollection("projects/sat-io/open-datasets/GABAM")

# gabam = gabam.filterDate('2015-01-01', '2015-12-31').mean()

# #clip lc
# lc = lc.filterDate('2015-01-01', '2015-12-31').max().clip(territory22.union())
# lc = lc.select('LC_Type1')


#union to a single feature for the table
# aoi = table.union()
# dnbr_ind = ee.ImageCollection("users/spotter/fire_cnn/all_dnbr_individual")
# dnbr_ind = dnbr_ind.max().clip(aoi)

#geometry is now aoi
# geometry = aoi


def to_short(image): 
    image1 = image.select('pre_7_NBR').cast({'pre_7_NBR': 'short'})
    image2 = image.select('pos_7_NBR').cast({'pos_7_NBR': 'short'})
    image3 = image.select('pre_8_NBR').cast({'pre_8_NBR': 'short'})
    image4 = image.select('pos_8_NBR').cast({'pos_8_NBR': 'short'})

    image = image.addBands(image1).addBands(image2).addBands(image3).addBands(image4)
    return image

def add_bands(image):
    image = image.addBands(ee.Image(ee.Date(image.get('system:time_start')).getRelative('day','year').add(1)).clamp(1,366)).updateMask(image.select('FireMask').gte(7))
    return image.updateMask(image.select('constant').gt(60))

def to_year(f):
    return f.set({'Year': year})


#no loop through the sub grid cells and get the area of each
all_territories = ee.List(territory22.distinct(["ID"]).aggregate_array("ID")).getInfo()

# # all_territories = [46, 47]
# all_territories = [31, 30, 29, 28,
#                   48, 47, 46, 45,
#                   69, 68, 67, 66,
#                   92, 91, 90, 89]

# # all_territories = [31]
# all_territories = [14, 18, 19, 21, 22, 23, 25, 26, 28]
final = []

# all_territories = [101]
for this_id in all_territories:
    
    table = territory22.filter(ee.Filter.eq('ID', this_id))

    
    lfdb = ee.FeatureCollection("users/spotter/fire_cnn/raw/ak_ca_1985")
    viirs = ee.Image("users/spotter/fire_cnn/ak_ca_viirs_2015")

    buffer = ee.FeatureCollection("users/spotter/fire_cnn/buffered_polys/2015")
    lc = ee.ImageCollection("MODIS/061/MCD12Q1")
        # final2 = ee.Image("users/spotter/fire_cnn/om_com_calc/2015"),
    water = ee.ImageCollection("JRC/GSW1_4/YearlyHistory")

    water = water.filterDate('2015-01-01', '2015-12-31').max().select('waterClass').clip(table)
    water = water.updateMask(water.eq(3))

    gabam = ee.ImageCollection("projects/sat-io/open-datasets/GABAM")

    gabam = gabam.filterDate('2015-01-01', '2015-12-31').mean()

    lc = lc.filterDate('2015-01-01', '2015-12-31').max().clip(table)
    lc = lc.select('LC_Type1')


    #clip viirs
    viirs = viirs.clip(table).select(['b1'], ['constant'])

    #get modis active fire and merge with viirs
    Terra_Fire = ee.ImageCollection("MODIS/061/MOD14A1").filterBounds(table)
    Aqua_Fire = ee.ImageCollection("MODIS/061/MYD14A1").filterBounds(table)


    
    mod_fire = Terra_Fire.merge(Aqua_Fire).filterBounds(table).filterDate(str(year) + '-01-01', str(year) + '-12-31')
    
    mod_fire =  mod_fire.map(add_bands)

    mod_fire = mod_fire.select('constant').max().clip(table)

    #merge modis and viirs
    mod_viirs = ee.ImageCollection([viirs, mod_fire]).max()
    # Map.addLayer(mod_viirs, {palette: 'yellow'}, 'MODIS-VIIRS')


    buffer = buffer.map(to_year)

    buff_img = buffer.reduceToImage(
      properties= ['Year'],
      reducer= ee.Reducer.max()).clip(table).select(['max'], ['constant'])

    #merge all
    final = ee.ImageCollection([mod_viirs, buff_img]).max().updateMask(lc.neq(12).And(lc.neq(16)))

    # predictions = ee.ImageCollection('users/spotter/fire_cnn/mtbs_predictions/ak_compare_450_64').max()
   # predictions = ee.ImageCollection('users/spotter/fire_cnn/mtbs_predictions/nwt_compare_450_64').max()
    predictions = ee.Image('users/spotter/fire_cnn/mtbs_predictions/ak_compare_combined_test')

    predictions2 = ee.Image('users/spotter/fire_cnn/mtbs_predictions/ak_compare_combined_median')

 # thresholded = predictions.select('prediction').updateMask(final).updateMask(predictions.select('prediction').gte(0.5))
    # thresholded = thresholded.updateMask(water.unmask().Not())

    gabam = gabam.updateMask(predictions2)

    gabam = gabam.updateMask(final).updateMask(lc.neq(12).And(lc.neq(16)))
    lfdb = ee.FeatureCollection('users/spotter/fire_cnn/raw/ak_ca_1985')

  
    lfdb = lfdb.filter(ee.Filter.eq("Year", year))

    #make image of fire databases
    lfdb_img = lfdb.reduceToImage(
      properties= ['Year'],
      reducer= ee.Reducer.max()).clip(table).select(['max'], ['constant'])

    lfdb_img = lfdb_img.updateMask(water.unmask().Not())
   

    #now loop through the bands and get the stats
    all_bands = predictions.bandNames().getInfo()
    
    #empty list for final
    final = []
    
    for band in all_bands:
        
        pred_sub = predictions.select(band)
    
        #get intersection of us and mtbs
        us_int = lfdb_img.updateMask(pred_sub).select(['constant'], ['us_int'])

        #get omission of us and mtbs (mtbs has fire we do not)
        us_om = lfdb_img.updateMask(pred_sub.unmask().Not()).select(['constant'], ['us_om'])

        #get comission of us and mtbs  (we have fire but mtbs does not)
        us_com = pred_sub.updateMask(lfdb_img.unmask().Not()).select([band], ['us_com'])

        #get intersection of gabam and mtbs
        gabam_int = lfdb_img.updateMask(gabam).select(['constant'], ['gabam_int'])

        #get omission of gabam and mtbs
        gabam_om = lfdb_img.updateMask(gabam.unmask().Not()).select(['constant'], ['gabam_om'])

        #get comission of gabam and mtbs
        gabam_com = gabam.updateMask(lfdb_img.unmask().Not()).select(['b1'], ['gabam_com'])

        #now get areas and convert to feature collection and send out to csv
        #now get the areas
        pa = ee.Image.pixelArea()

        #areas are in m2

        img_area = pa.updateMask(lfdb_img).reduceRegion(reducer= ee.Reducer.sum(),geometry= table,scale= 30,tileScale= 16,crs= 'EPSG:3413',bestEffort=False,maxPixels= 1e13).get('area')

        us_int_area = pa.updateMask(us_int).reduceRegion(reducer= ee.Reducer.sum(),geometry= table,scale= 30,tileScale= 16,crs= 'EPSG:3413',bestEffort=False,maxPixels= 1e13).get('area')

        us_om_area = pa.updateMask(us_om).reduceRegion(reducer= ee.Reducer.sum(),geometry= table,scale= 30,tileScale= 16,crs= 'EPSG:3413',bestEffort=False,maxPixels= 1e13).get('area')

        us_com_area = pa.updateMask(us_com).reduceRegion(reducer= ee.Reducer.sum(),geometry= table,scale= 30,tileScale= 16,crs= 'EPSG:3413',bestEffort=False,maxPixels= 1e13).get('area')

        gabam_int_area = pa.updateMask(gabam_int).reduceRegion(reducer= ee.Reducer.sum(),geometry= table,scale= 30,tileScale= 16,crs= 'EPSG:3413',bestEffort=False,maxPixels= 1e13).get('area')

        gabam_om_area = pa.updateMask(gabam_om).reduceRegion(reducer= ee.Reducer.sum(),geometry= table,scale= 30,tileScale= 16,crs = 'EPSG:3413',bestEffort=False,maxPixels= 1e13).get('area')

        gabam_com_area = pa.updateMask(gabam_com).reduceRegion(reducer= ee.Reducer.sum(),geometry= table,scale= 30,tileScale= 16,crs= 'EPSG:3413',bestEffort=False,maxPixels= 1e13).get('area')


        us_int_feat = ee.Feature(None, {'Year': year, 'ID': this_id, 'Class': 'Us_Int', 'Area': us_int_area, 'Band': band})
        us_om_feat = ee.Feature(None, {'Year': year, 'ID': this_id, 'Class': 'Us_Om', 'Area': us_om_area,'Band': band})
        us_com_feat = ee.Feature(None, {'Year': year, 'ID': this_id, 'Class': 'Us_Com', 'Area': us_com_area,'Band': band})
        gabam_int_feat = ee.Feature(None, {'Year': year, 'ID': this_id, 'Class': 'Gabam_Int', 'Area': gabam_int_area,'Band': band})
        gabam_om_feat = ee.Feature(None, {'Year': year, 'ID': this_id, 'Class': 'Gabam_Om', 'Area': gabam_om_area,'Band': band})
        gabam_com_feat = ee.Feature(None, {'Year': year, 'ID': this_id, 'Class': 'Gabam_Com', 'Area': gabam_com_area,'Band': band})
        fire_feat = ee.Feature(None, {'Year': year, 'ID': this_id, 'Class': 'Fire_Area', 'Area': img_area,'Band': band})

        final_sub = ee.FeatureCollection([us_int_feat, us_om_feat, us_com_feat, gabam_int_feat, gabam_om_feat, gabam_com_feat, fire_feat])
        final.append(final_sub)



    final = ee.FeatureCollection(final).flatten()

    print('Downloading ' +  'om_com_ak_compare_combined_median_' + str(year) + '_' + str(this_id))
    task = ee.batch.Export.table.toCloudStorage(collection = final, 
                                description = 'om_com_ak_compare_combined_median_' + str(year) + '_' + str(this_id),
                                bucket =  'smp-scratch', 
                                fileFormat = 'CSV')

    task.start()


    
    
    
    
    



In [32]:
img_area.getInfo()

734489607.2222935

In [34]:
us_om_area.getInfo()

90701053.20710452

In [30]:
table.getInfo()

{'type': 'FeatureCollection',
 'columns': {'ID': 'Float',
  'count': 'Long',
  'label': 'Long',
  'system:index': 'String'},
 'version': 1686612996021012,
 'id': 'users/spotter/fire_cnn/raw/ak_cent_grid',
 'properties': {'system:asset_size': 99109},
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-154.0934900680627, 64.99016751559853],
      [-154.21639236075052, 64.97209113490187],
      [-154.3391112062152, 64.95390449501129],
      [-154.4616456677306, 64.93560780606417],
      [-154.5839953382023, 64.91720137489199],
      [-154.7061593724703, 64.8986855242079],
      [-154.82813703240978, 64.8800604983744],
      [-154.94992776125392, 64.8613265737754],
      [-155.07153091330719, 64.84248410669981],
      [-155.027227264007, 64.7907643688918],
      [-154.98311086855963, 64.73904022474431],
      [-154.93918057032283, 64.68731167424339],
      [-154.89543502938736, 64.63557887574366],
      [-154.85187335502448, 64.58384196942337],
     